In [1]:
include("lib/all.jl");

In [2]:
test_mov()

705537

In [3]:
test_acc()

839063

In [16]:
particles = make_particles(2^20);
t = Tree(particles, 12);

In [17]:
@time group!(t);

  0.145144 seconds (4 allocations: 160 bytes)


In [18]:
@time inform!(t);

  0.077621 seconds (1.10 M allocations: 16.856 MB)


In [19]:
t.num_nodes_used

276822

In [20]:
test_exp();

dx = 7.0
dx3 = 343.0
(get_accel_from_node(n1,n1.x,n1.y,n1.z))[1] = 0.017640358229920005
(get_accel_from_node(n1,tx,ty,tz))[1] = 0.028808177611957104
((dx * m) / dr2) / sqrt(dr2) = 0.025613908599594084
px = 0.017640358229920005
pxx = 0.0043400881359327
pxxx = 0.001506326885888231
pxy = -0.0019600398033244454


In [21]:
test_in_cell_mass(t)

In [22]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [23]:
@time interact!(t, 0.5, ax, ay, az);

  1.653778 seconds (4 allocations: 160 bytes)


In [24]:
@time collect!(t, ax,ay,az)

  0.406861 seconds (39.79 M allocations: 607.768 MB, 16.19% gc time)


In [25]:
perf(t.particles, ax, 1000, 1.0e-178)

1.0635106972472763